**<h1 align="center">Image Exploration</h1>**

## 🖼️ **Image Viewer for Paradise Masks**
- **Overlay Visualization**: View segmentation overlays with dual opacity lung zones
- **Mask Analysis**: Examine binary lung segmentation masks
- **Flexible Display**: Customizable grid layout and filtering options
- **Random Sampling**: Option to randomly sample images for quick overview


In [ ]:
# 🌟 Global Configuration Variables

# CSV Files
CSV_FOLDER = "/home/pyuser/wkdir/CSI-Predictor/data/Paradise_CSV/"
CSV_LABELS_FILE = "Labeled_Data_RAW_Sample.csv"
CSV_SEPARATOR = ";"
IMPORT_COLUMNS = []
CHUNK_SIZE = 50000

# Download parameters  
DOWNLOAD_PATH = '/home/pyuser/wkdir/CSI-Predictor/data/Paradise_Test_DICOMs'
IMAGES_PATH = '/home/pyuser/wkdir/CSI-Predictor/data/Paradise_Test_Images'
EXPORT_METADATA = True
ARCHIMED_METADATA_FILE = 'DICOM_Metadata.csv'
CONVERT = True

# Conversion parameters
BATCH_SIZE = 50
BIT_DEPTH = 8
CREATE_SUBFOLDERS = False
DELETE_DICOM = True
MONOCHROME = 1

# Enhanced Parameters
TARGET_SIZE = (518, 518)
PRESERVE_ASPECT_RATIO = True

# 🖼️ IMAGE EXPLORATION SETTINGS
FOLDER_PATH = '/home/pyuser/data/Paradise_Masks'

print("🌟 Image Exploration configuration loaded!")
print(f"📁 Target folder: {FOLDER_PATH}")
print(f"📐 Target size: {TARGET_SIZE}")


In [ ]:
# Core dependencies
import os
import glob
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2
from IPython.display import display, HTML

# Colors for output
ANSI = {
    'R': '\033[91m', 'G': '\033[92m', 'B': '\033[94m', 'Y': '\033[93m',
    'W': '\033[0m', 'M': '\033[95m', 'C': '\033[96m'
}

print(f"{ANSI['G']}✅ Core dependencies loaded{ANSI['W']}")


In [ ]:
def display_images(folder_path, filter=None, max_columns=3, max_nb_images=None, randomized=False):
    """
    Display images from a folder in a grid layout with flexible filtering and sampling options.
    
    Parameters:
    -----------
    folder_path : str
        Path to the folder containing PNG files
    filter : str, optional
        Image filenames must contain this filter string if provided
    max_columns : int, default=3
        Maximum number of images per row
    max_nb_images : int, optional
        Maximum number of images to display (if set)
    randomized : bool, default=False
        If True, randomly sample images; if False, display in alphabetical order
    """
    
    print(f"{ANSI['C']}🖼️ Loading images from: {folder_path}{ANSI['W']}")
    
    # Check if folder exists
    if not os.path.exists(folder_path):
        print(f"{ANSI['R']}❌ Folder not found: {folder_path}{ANSI['W']}")
        return
    
    # Get all PNG files
    png_files = glob.glob(os.path.join(folder_path, "*.png"))
    
    # Apply filter if provided
    if filter:
        png_files = [f for f in png_files if filter in os.path.basename(f)]
        print(f"{ANSI['Y']}🔍 Filter applied: '{filter}' - Found {len(png_files)} matching files{ANSI['W']}")
    else:
        print(f"{ANSI['B']}📋 No filter applied - Found {len(png_files)} PNG files{ANSI['W']}")
    
    if len(png_files) == 0:
        print(f"{ANSI['Y']}⚠️ No images found matching criteria{ANSI['W']}")
        return
    
    # Sort or randomize files
    if randomized:
        random.shuffle(png_files)
        print(f"{ANSI['M']}🎲 Images randomized{ANSI['W']}")
    else:
        png_files.sort()
        print(f"{ANSI['G']}📚 Images sorted alphabetically{ANSI['W']}")
    
    # Limit number of images if specified
    if max_nb_images and max_nb_images < len(png_files):
        png_files = png_files[:max_nb_images]
        print(f"{ANSI['C']}✂️ Limited to {max_nb_images} images{ANSI['W']}")
    
    # Calculate grid dimensions
    num_images = len(png_files)
    num_rows = (num_images + max_columns - 1) // max_columns  # Ceiling division
    
    print(f"{ANSI['G']}📐 Display grid: {num_rows} rows × {max_columns} columns (showing {num_images} images){ANSI['W']}")
    
    # Create matplotlib figure
    fig, axes = plt.subplots(num_rows, max_columns, figsize=(4*max_columns, 4*num_rows))
    
    # Handle single row case
    if num_rows == 1:
        axes = axes.reshape(1, -1) if num_images > 1 else np.array([[axes]])
    elif max_columns == 1:
        axes = axes.reshape(-1, 1)
    
    # Display images
    for idx, img_path in enumerate(png_files):
        row = idx // max_columns
        col = idx % max_columns
        
        try:
            # Load and display image
            img = Image.open(img_path)
            axes[row, col].imshow(img, cmap='gray' if img.mode == 'L' else None)
            
            # Set title (filename without extension)
            filename = os.path.splitext(os.path.basename(img_path))[0]
            axes[row, col].set_title(filename, fontsize=10, pad=10)
            axes[row, col].axis('off')
            
        except Exception as e:
            # Handle broken images
            axes[row, col].text(0.5, 0.5, f'Error loading\n{os.path.basename(img_path)}', 
                               ha='center', va='center', transform=axes[row, col].transAxes)
            axes[row, col].axis('off')
            print(f"{ANSI['R']}❌ Error loading {img_path}: {e}{ANSI['W']}")
    
    # Hide empty subplots
    for idx in range(num_images, num_rows * max_columns):
        row = idx // max_columns
        col = idx % max_columns
        axes[row, col].axis('off')
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
    
    # Summary
    print(f"\n{ANSI['G']}✅ Successfully displayed {num_images} images{ANSI['W']}")
    if filter:
        print(f"{ANSI['C']}🏷️ Filter: '{filter}'{ANSI['W']}")
    if max_nb_images:
        print(f"{ANSI['Y']}📊 Limited to: {max_nb_images} images{ANSI['W']}")
    if randomized:
        print(f"{ANSI['M']}🎲 Randomized sampling enabled{ANSI['W']}")

print(f"{ANSI['G']}✅ Image display function loaded{ANSI['W']}")


## 🎨 Overlay Images Visualization

Display overlay images showing:
- **Dark Red**: Lung segmentation zones (Fill: 50%, Border: 75%)
- **Green**: Segmentation corner brackets (exact boundaries)
- **Cyan**: Final resize crop rectangle (1px contour)
- **Black Legend**: Explanation with 75% opacity background


In [ ]:
# 🎨 Display Overlay Images
display_images(
    folder_path=FOLDER_PATH,
    filter="overlay",
    max_columns=3
)


## 🎭 Binary Mask Images

Display binary segmentation masks showing:
- **White**: Detected lung tissue areas
- **Black**: Background (non-lung areas)
- **Clean Binary**: Simple contours for analysis


In [ ]:
# 🎭 Display Binary Mask Images
display_images(
    folder_path=FOLDER_PATH,
    filter="mask",
    max_columns=3
)
